In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import urllib.request, urllib.parse, urllib.error
# import
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

In [2]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [3]:
path_data_science = []
for i in range(1, 501):
    
    path = 'https://www.seek.com.au/Data-Science-jobs/in-All-Australia?page=' + str(i)
    path_data_science.append(path)
    
for i in range(1, 501):
    
    path = 'https://www.seek.com.au/Data-Scientist-jobs/in-All-Australia?page=' + str(i)
    path_data_science.append(path)
    
for i in range(1, 501):
    
    path = 'https://www.seek.com.au/Machine-Learning-jobs/in-All-Australia?page=' + str(i)
    path_data_science.append(path)

In [4]:
urls_data_science = []
dupe_count = 0
for path in log_progress(path_data_science):
    # load the page
    page = requests.get(path)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # grab title url
    titles = soup.find_all('a', {'data-automation': 'jobTitle'}) 
    for title in titles:
        url = 'https://www.seek.com.au' + title.get('href')
        if url not in urls_data_science:
            urls_data_science.append(url)
        else:
            dupe_count+=1

print('duplicates', dupe_count)


duplicates 480


In [5]:
len(urls_data_science)

858

In [6]:
urls_data_science_df = pd.DataFrame(urls_data_science, columns = ['urls'])

In [7]:
urls_data_science_df.to_csv('./Q2_data_scraping/seek_urls_data_science.csv')

### Scrape all Data Science jobs

In [8]:
urls_data_science_df = pd.read_csv('./Q2_data_scraping/seek_urls_data_science.csv')

In [10]:
urls_data_science = urls_data_science_df.urls.values.tolist()

In [11]:
title_list = []
company_list = []
date_list = []
location_list = []
work_type_list = []
job_type_list = []
description_list = []
url_list = []

for url in log_progress(urls_data_science):
    detailed_path = url
    page = requests.get(detailed_path)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    ## URL
    url_list.append(url)
    
    ## TITLES
    title_object = soup.find('span', {'data-automation': 'job-detail-title'})
    title = None
    try:
        title = title_object.text
    except:
        pass
    
    title_list.append(title)
    
    ## COMPANY
    company = None

    try:
        company_object = soup.find('span', {'data-automation': 'advertiser-name'})
        company = company_object.text

    except:
        pass
    
    company_list.append(company)
    
    
    ## DATE
    date_object = soup.find('dd', {'data-automation': 'job-detail-date'})
    date = None

    try:
        date = date_object.text
    except:
        pass
    
    date_list.append(date)
    
    ## LOCATION
    location = None
    try:
        info_header = soup.find('section', {'aria-labelledby': 'jobInfoHeader'})
        location_object = info_header.find('strong', {'class': 'lwHBT6d'})
        location= location_object.text
    except:
        pass
    
    location_list.append(location)
    
    ## WORK TYPE
    work_type = None

    try:
        work_type_object = soup.find('dd',{'data-automation': 'job-detail-work-type'})
        work_type = work_type_object.text
    except:
        pass
    
    work_type_list.append(work_type)
    
    ## JOB TYPE
    job_type = None

    try:
        info_header = soup.find('section', {'aria-labelledby': 'jobInfoHeader'})
        job_type = info_header.find_all('dd')[-1].find('strong', {'class': 'lwHBT6d'}).text
    except:
        pass

    job_type_list.append(job_type)
    
    ## DESCRIPTION
    description = None

    try:
        description_object = soup.find('div', {'data-automation': 'jobDescription'})
        for p in description_object.find_all('p'):
            if description == None:
                description = p.text
            else: 
                description = description + ' ' + p.text
    except:
        pass


    try:
        description_object = soup.find('div', {'data-automation': 'jobDescription'})
        for li in description_object.find_all('li'):
            if description == None:
                description = li.text
            else: 
                description = description + ' ' + li.textb
    except:
        pass
    
    description_list.append(description)
    

seek_data_science_df = pd.DataFrame({'url': url_list,
                               'job_title': title_list,
                               'company': company_list,
                               'date': date_list,
                               'location': location_list,
                               'employment_type': work_type_list,
                               'field': job_type_list,
                               'description': description_list})

seek_data_science_df.to_csv('./Q2_data_scraping/seek_data_science.csv')


In [13]:
len(seek_data_science_df)


858

### Data Analytic jobs

In [15]:
path_data_analytic = []
for i in range(1, 1001):
    path = 'https://www.seek.com.au/Data-Analytic-jobs/in-All-Australia?page=' + str(i)
    path_data_analytic.append(path)
    
    

In [17]:
urls_data_analytic = []
dupe_count = 0
for path in log_progress(path_data_analytic):
    # load the page
    page = requests.get(path)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # grab title url
    titles = soup.find_all('a', {'data-automation': 'jobTitle'}) 
    for title in titles:
        url = 'https://www.seek.com.au' + title.get('href')
        if url not in urls_data_science and url not in urls_data_analytic:
            urls_data_analytic.append(url)
        else:
            dupe_count+=1

print('duplicates', dupe_count)


duplicates 413


In [20]:
len(urls_data_analytic)

1379

In [21]:
urls_data_analytic_df = pd.DataFrame(urls_data_analytic, columns = ['urls'])

In [22]:
urls_data_analytic_df.to_csv('./Q2_data_scraping/seek_urls_data_analytic.csv')

### Scrape all data analytic jobs

In [23]:
urls_data_analytic_df = pd.read_csv('./Q2_data_scraping/seek_urls_data_analytic.csv')

In [24]:
urls_data_analytic = urls_data_analytic_df.urls.values.tolist()

In [25]:
title_list = []
company_list = []
date_list = []
location_list = []
work_type_list = []
job_type_list = []
description_list = []
url_list = []

for url in log_progress(urls_data_analytic):
    detailed_path = url
    page = requests.get(detailed_path)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    ## URL
    url_list.append(url)
    
    ## TITLES
    title_object = soup.find('span', {'data-automation': 'job-detail-title'})
    title = None
    try:
        title = title_object.text
    except:
        pass
    
    title_list.append(title)
    
    ## COMPANY
    company = None

    try:
        company_object = soup.find('span', {'data-automation': 'advertiser-name'})
        company = company_object.text

    except:
        pass
    
    company_list.append(company)
    
    
    ## DATE
    date_object = soup.find('dd', {'data-automation': 'job-detail-date'})
    date = None

    try:
        date = date_object.text
    except:
        pass
    
    date_list.append(date)
    
    ## LOCATION
    location = None
    try:
        info_header = soup.find('section', {'aria-labelledby': 'jobInfoHeader'})
        location_object = info_header.find('strong', {'class': 'lwHBT6d'})
        location= location_object.text
    except:
        pass
    
    location_list.append(location)
    
    ## WORK TYPE
    work_type = None

    try:
        work_type_object = soup.find('dd',{'data-automation': 'job-detail-work-type'})
        work_type = work_type_object.text
    except:
        pass
    
    work_type_list.append(work_type)
    
    ## JOB TYPE
    job_type = None

    try:
        info_header = soup.find('section', {'aria-labelledby': 'jobInfoHeader'})
        job_type = info_header.find_all('dd')[-1].find('strong', {'class': 'lwHBT6d'}).text
    except:
        pass

    job_type_list.append(job_type)
    
    ## DESCRIPTION
    description = None

    try:
        description_object = soup.find('div', {'data-automation': 'jobDescription'})
        for p in description_object.find_all('p'):
            if description == None:
                description = p.text
            else: 
                description = description + ' ' + p.text
    except:
        pass


    try:
        description_object = soup.find('div', {'data-automation': 'jobDescription'})
        for li in description_object.find_all('li'):
            if description == None:
                description = li.text
            else: 
                description = description + ' ' + li.text
    except:
        pass
    
    description_list.append(description)
    
    
seek_data_analytic_df = pd.DataFrame({'url': url_list,
                               'job_title': title_list,
                               'company': company_list,
                               'date': date_list,
                               'location': location_list,
                               'employment_type': work_type_list,
                               'field': job_type_list,
                               'description': description_list})

seek_data_analytic_df.to_csv('./Q2_data_scraping/seek_data_analytic.csv')

In [26]:
seek_data_analytic_df.shape

(1379, 8)

### Try scraping detailed page

In [ ]:
detailed_path = 'https://www.seek.com.au/job/37457759?type=standard'
page = requests.get(detailed_path)
soup = BeautifulSoup(page.content, 'html.parser')





#### Title

In [ ]:
# TITLE
title_object = soup.find('span', {'data-automation': 'job-detail-title'})
title = ''
try:
    title = title_object.text
except:
    title = None

In [ ]:
title

#### Company type

In [ ]:
company = ''

try:
    company_object = soup.find('span', {'data-automation': 'advertiser-name'})
    company = company_object.text
    
except:
    company = None
    

In [ ]:
company

#### Date

In [ ]:
date_object = soup.find('dd', {'data-automation': 'job-detail-date'})
date = ''

try:
    date = date_object.text
except:
    date = None

In [ ]:
date

#### Location

In [ ]:
location = ''
try:
    info_header = soup.find('section', {'aria-labelledby': 'jobInfoHeader'})
    location_object = info_header.find('strong', {'class': 'lwHBT6d'})
    location= location_object.text
except:
    location = None

In [ ]:
location

#### Work type

In [ ]:
work_type = ''

try:
    work_type_object = soup.find('dd',{'data-automation': 'job-detail-work-type'})
    work_type = work_type_object.text
except:
    work_type = None

In [ ]:
work_type_object.text

#### Job type

In [ ]:
job_type = ''

try:
    info_header = soup.find('section', {'aria-labelledby': 'jobInfoHeader'})
    job_type = info_header.find_all('dd')[-1].find('strong', {'class': 'lwHBT6d'}).text
except:
    job_type = None

    

In [ ]:
job_type


#### Job description

In [ ]:
description = ''

description_object = soup.find('div', {'data-automation': 'jobDescription'})

In [ ]:
description = None

try:
    description_object = soup.find('div', {'data-automation': 'jobDescription'})
    for p in description_object.find_all('p'):
        if description == None:
            description = p.text
        else: 
            description = description + ' ' + p.text
except:
    pass
    
    
try:
    description_object = soup.find('div', {'data-automation': 'jobDescription'})
    for li in description_object.find_all('li'):
        if description == None:
            description = li.text
        else: 
            description = description + ' ' + li.text
except:
    pass

In [ ]:
description